In [253]:
from __future__ import print_function, division, absolute_import

import os

%matplotlib inline
import matplotlib
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.io.ascii as at
import astropy.io.fits as fits
import astropy.units as u
from scipy.interpolate import interp1d
from cycler import cycler

In [254]:
cd /Users/amandaash/Desktop/Research/scripts/

/Users/amandaash/Desktop/Research/scripts


In [255]:
import spectra_functions
import pyspeckit

In [256]:
from PHEW import EqW
import pyspeckit as p

In [ ]:
name = 'A575'
fname =  '/Users/amandaash/Desktop/Research/data/CSCU_reductions/20180116/finals/trim.{0}.fits'.format(name)
eperc = EqW.measure_equivalent_width(str(fname),
                                     6550,6576,6560,6566,
                                     1000,xunit="Angstrom",to_plot=True,
                                     filebase="/Users/amandaash/Desktop/Research/plots/figures/{0}_{1}.pdf".format(name,20180116))
#["eqw16"], ["eqw_med"], ["eqw84"] = eperc

In [259]:
#use PHEW on each target exposure 

import csv
import glob as glob
EW = []
with open('/Users/amandaash/Desktop/Research/EW.csv', 'r') as csvFile:
    EW_data = csv.reader(csvFile)
    for row in EW_data:
    
        EW.append(list(row))
csvFile.close()


import csv
import glob as glob
target_list = []
with open('/Users/amandaash/Desktop/Research/data/Target_names.csv', 'r') as csvFile:
    target_data = csv.reader(csvFile)
    for row in target_data:
        target_list.append(row[1])
csvFile.close()

targets = target_list[2:]


for target in targets:
    date_array = []
    EqW_array = []
    lower_error_array = []
    upper_error_array = []
    for n in EW[1:]:
        name = n[0].split('[')[1].split(']')[0]
        if name == "'" + target + "'":
            date = int(n[1])
            if date == 20180111:
                date_array.append(1)
            if date == 20180112:
                date_array.append(2)
            if date == 20180113:
                date_array.append(3)
            if date == 20180114:
                date_array.append(4)
            if date == 20180115:
                date_array.append(5)
            if date == 20180116:
                date_array.append(6)
            if date == 20180117:
                date_array.append(7)
            if date == 20180118:
                date_array.append(8)
            if date == 20180119:
                date_array.append(9)
            if date == 20180120:
                date_array.append(10)
            if date == 20180121:
                date_array.append(11)
            if date == 20180122:
                date_array.append(12)
            if date == 20180123:
                date_array.append(13)
            if date == 20180124:
                date_array.append(14)
            if date == 20180125:
                date_array.append(15)
            if date == 20180126:
                date_array.append(16)
            if date == 20180127:
                date_array.append(17)
            if date == 20180128:
                date_array.append(18)
            if date == 20180129:
                date_array.append(19)
            if date == 20180130:
                date_array.append(20)
            if date == 20180131:
                date_array.append(21)
            if date == 20180201:
                date_array.append(22)
            if date == 20180202:
                date_array.append(23)
            if date == 20180203:
                date_array.append(24)
            if date == 20180204:
                date_array.append(25)
            if date == 20180205:
                date_array.append(26)
            if date == 20180206:
                date_array.append(27)
            if date == 20180207:
                date_array.append(28)
            if date == 20180208:
                date_array.append(29)
            if date == 20180209:
                date_array.append(30)
            EqW = float(n[2].split(' ')[1])
            #print(float(EqW))
            perc16 = float(n[2].split(' ')[0].split('[')[1])
            perc84 = float(n[2].split(' ')[2].split(']')[0])
            lower_uncertainty = EqW - perc16
            lower_error_array.append(lower_uncertainty)
            upper_uncertainty = perc84 - EqW
            upper_error_array.append(upper_uncertainty)
            EqW_array.append(EqW)
        else:
            continue
    plt.figure(figsize=(10,8))
    ax = plt.subplot(111)
    v = [lower_error_array, upper_error_array]
    days = np.arange(20180110,20180131,1)
    np.append(days, np.arange(20180200,20180209,1))
    #print(v.shape)
    ax.errorbar(date_array,EqW_array, yerr = v, fmt = 'o', color = 'b')
    #ax.plot(date_array,EqW_array, linestyle = 'dotted')
    ax.set_title(target)
    ax.set_ylabel(r'$H_{\alpha}\ EW\ (\AA)$')
    ax.set_xlabel('time (days)')
    #ax.set_xticklabels(days,rotation = 90)
    plt.savefig('/Users/amandaash/Desktop/Research/plots/EW_variability_61919/{0}_EW_variability.pdf'.format(target))
    plt.close()




           

